# Importacion y generacion de matriz para algoritmos

In [44]:
import os
import numpy as np
import json
import time
import itertools
from itertools import combinations, permutations
import heapq
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary, LpStatus, value, PULP_CBC_CMD
import random

In [45]:
"""
Se genera de manera random una nueva matriz de trabajo y se guarda en un archivo
tipo .npy
Nota: Si se requiere volver a generar una nueva matriz random, por favor eliminar el archivo
para que lo genere de nuevo, de lo contrario trabajara con el que ya existe.

"""
ruta_npy = os.path.join("data" ,"matriz_trabajo.npy")
matriz_actores = []

try:
    matriz_actores_original = np.array([
        [1,1,1,1,1,0,0,0,0,0], [0,0,1,1,1,0,0,0,0,0], [0,1,0,0,1,0,1,0,0,0],
        [1,1,0,0,0,0,1,1,0,0], [0,1,0,1,0,0,0,1,0,0], [1,1,0,1,1,0,0,0,0,0],
        [1,1,0,1,1,0,0,0,0,0], [1,1,0,0,0,1,0,0,0,0], [1,1,0,1,0,0,0,0,0,0],
        [1,1,0,0,0,1,0,0,1,0], [1,1,1,0,1,0,0,1,0,0], [1,1,1,1,0,1,0,0,0,0],
        [1,0,0,1,1,0,0,0,0,0], [1,0,1,0,0,1,0,0,0,0], [1,1,0,0,0,0,1,0,0,0],
        [0,0,0,1,0,0,0,0,0,1], [1,0,1,0,0,0,0,0,0,0], [0,0,1,0,0,1,0,0,0,0],
        [1,0,1,0,0,0,0,0,0,0], [1,0,1,1,1,0,0,0,0,0], [0,0,0,0,0,1,0,1,0,0],
        [1,1,1,1,0,0,0,0,0,0], [1,0,1,0,0,0,0,0,0,0], [0,0,1,0,0,1,0,0,0,0],
        [1,1,0,1,0,0,0,0,0,1], [1,0,1,0,1,0,0,0,1,0], [0,0,0,1,1,0,0,0,0,0],
        [1,0,0,1,0,0,0,0,0,0], [1,0,0,0,1,1,0,0,0,0], [1,0,0,1,0,0,0,0,0,0],
    ])
    matriz_actores = np.load(ruta_npy)
except FileNotFoundError:
    filas = np.random.randint(10,11)
    columnas = np.random.randint(6,9)
    matriz_actores = np.random.randint(2, size=(filas, columnas))
    np.save(ruta_npy, matriz_actores)

print(matriz_actores)
print()
print(matriz_actores_original)

[[0 0 0 1 1 0 0 0]
 [0 1 0 1 1 1 1 1]
 [1 0 0 0 0 1 1 1]
 [1 1 1 1 1 1 0 1]
 [1 0 0 0 0 0 1 1]
 [0 1 1 0 0 1 0 0]
 [0 0 0 0 1 1 1 1]
 [0 0 0 1 1 1 0 1]
 [1 0 1 0 1 1 0 0]
 [0 1 0 1 1 1 0 0]]

[[1 1 1 1 1 0 0 0 0 0]
 [0 0 1 1 1 0 0 0 0 0]
 [0 1 0 0 1 0 1 0 0 0]
 [1 1 0 0 0 0 1 1 0 0]
 [0 1 0 1 0 0 0 1 0 0]
 [1 1 0 1 1 0 0 0 0 0]
 [1 1 0 1 1 0 0 0 0 0]
 [1 1 0 0 0 1 0 0 0 0]
 [1 1 0 1 0 0 0 0 0 0]
 [1 1 0 0 0 1 0 0 1 0]
 [1 1 1 0 1 0 0 1 0 0]
 [1 1 1 1 0 1 0 0 0 0]
 [1 0 0 1 1 0 0 0 0 0]
 [1 0 1 0 0 1 0 0 0 0]
 [1 1 0 0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 0 0 1]
 [1 0 1 0 0 0 0 0 0 0]
 [0 0 1 0 0 1 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0]
 [1 0 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 1 0 0]
 [1 1 1 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0]
 [0 0 1 0 0 1 0 0 0 0]
 [1 1 0 1 0 0 0 0 0 1]
 [1 0 1 0 1 0 0 0 1 0]
 [0 0 0 1 1 0 0 0 0 0]
 [1 0 0 1 0 0 0 0 0 0]
 [1 0 0 0 1 1 0 0 0 0]
 [1 0 0 1 0 0 0 0 0 0]]


# Algoritmo Fuerza Bruta SIN PuLP

In [43]:
cantidad_tomas = matriz_actores.shape[0]
cantidad_actores = matriz_actores.shape[1]
maximo_tomas_por_dia = 6

dias_a_probar_fuerza_bruta = 10

tiempo_maximo_ejecucion_por_ciclo = 3600
ruta_archivo_checkpoint = 'data/fuerza_bruta_checkpoint.json'
ruta_archivo_terminado = 'data/fuerza_bruta_terminada.flag'

def calcular_costo_planificacion(planificacion_por_dias, matriz_actores_global):
    costo_total_actores = 0
    for dia_actual, tomas_del_dia_actual in planificacion_por_dias.items():
        if not tomas_del_dia_actual:
            continue

        actores_necesarios_este_dia = np.zeros(matriz_actores_global.shape[1], dtype=bool)
        for indice_toma in tomas_del_dia_actual:
            actores_necesarios_este_dia = np.logical_or(actores_necesarios_este_dia, matriz_actores_global[indice_toma, :])

        costo_total_actores += np.sum(actores_necesarios_este_dia)
    return int(costo_total_actores)

def guardar_checkpoint(mejor_costo_actual, mejor_plan_actual, iteracion_actual, iteracion_inicio_global):
    datos_checkpoint = {
        'mejor_costo': mejor_costo_actual,
        'mejor_plan': mejor_plan_actual,
        'iteracion_actual_en_ciclo': iteracion_actual,
        'iteracion_para_reanudar': iteracion_inicio_global + iteracion_actual,
        'finalizado': False
    }
    with open(ruta_archivo_checkpoint, 'w') as f:
        json.dump(datos_checkpoint, f)
    print(f"\n<---***--- Checkpoint guardado en la iteración {iteracion_actual}. Mejor costo: {mejor_costo_actual} ---***--->")

def cargar_checkpoint():
    if os.path.exists(ruta_archivo_checkpoint):
        try:
            with open(ruta_archivo_checkpoint, 'r') as f:
                datos_checkpoint = json.load(f)
            print("<---***--- Checkpoint cargado exitosamente ---***--->")
            print(f"Mejor costo previo encontrado: {datos_checkpoint.get('mejor_costo', float('inf'))}")
            print(f"Reanudando desde la iteración global: {datos_checkpoint.get('iteracion_para_reanudar', 0)}")
            return datos_checkpoint
        except json.JSONDecodeError:
            print(f"Error al leer el archivo de checkpoint '{ruta_archivo_checkpoint}'")
            return None
    return None

def establecer_bandera_finalizado():
    with open(ruta_archivo_terminado, 'w') as f:
        f.write("FINALIZADO")
    print("\n<---***--- Algoritmo de Fuerza Bruta FINALIZADO completamente. ---***--->")

def esta_finalizado():
    return os.path.exists(ruta_archivo_terminado)

def imprimir_resultados_finales(costo, plan):
    print("\n--- Resultados finales ---")
    print(f"Mejor costo encontrado: {costo}")
    print("Planificación final:")
    if isinstance(plan, dict):
        for dia_res, tomas_indices in plan.items():
            print(f"  Día {dia_res}: Tomas {tomas_indices} (Cantidad de Tomas: {len(tomas_indices)})")
    else:
        print("No hay solución final o está guardado incorrectamente")

def ejecutar_fuerza_bruta(tiempo_maximo_ejecucion_por_ciclo):
    if esta_finalizado():
        print("El algoritmo ya finalizó completamente en anteriores ejecuciones, los resultados fueron: ")
        datos_checkpoint = cargar_checkpoint()
        if datos_checkpoint:
            imprimir_resultados_finales(datos_checkpoint['mejor_costo'], datos_checkpoint['mejor_plan'])
        return

    datos_checkpoint = cargar_checkpoint()
    if datos_checkpoint:
        mejor_costo_global = datos_checkpoint['mejor_costo']
        mejor_plan_global = datos_checkpoint['mejor_plan']
        iteracion_inicio_global = datos_checkpoint['iteracion_para_reanudar']
    else:
        mejor_costo_global = float('inf')
        mejor_plan_global = {}
        iteracion_inicio_global = 0

    print(f"\nIniciando Fuerza Bruta para {dias_a_probar_fuerza_bruta} días con {cantidad_tomas} tomas.")
    print(f"Mejor costo inicial: {mejor_costo_global} (si se cargó un checkpoint)")
    print(f"Tiempo máximo por ciclo de ejecución: {tiempo_maximo_ejecucion_por_ciclo} segundos")

    tiempo_inicio_ciclo_actual = time.time()

    todas_las_asignaciones = itertools.product(range(dias_a_probar_fuerza_bruta), repeat=cantidad_tomas)

    print(f"Avanzando el iterador hasta la iteración {iteracion_inicio_global}...")
    for _ in range(iteracion_inicio_global):
        try:
            next(todas_las_asignaciones)
        except StopIteration:
            print("\n<---***--- Todas las combinaciones posibles han sido exploradas. ---***--->")
            imprimir_resultados_finales(mejor_costo_global, mejor_plan_global)
            establecer_bandera_finalizado()
            guardar_checkpoint(mejor_costo_global, mejor_plan_global, 0, iteracion_inicio_global)
            return

    contador_iteraciones_ciclo = 0

    for asignacion_actual_tupla in todas_las_asignaciones:
        contador_iteraciones_ciclo += 1
        iteracion_global_actual = iteracion_inicio_global + contador_iteraciones_ciclo

        if (time.time() - tiempo_inicio_ciclo_actual) > tiempo_maximo_ejecucion_por_ciclo:
            print(f"\n--- Límite de tiempo ({tiempo_maximo_ejecucion_por_ciclo}s) alcanzado en este ciclo. Guardando checkpoint. ---")
            guardar_checkpoint(mejor_costo_global, mejor_plan_global, contador_iteraciones_ciclo, iteracion_inicio_global)
            imprimir_resultados_finales(mejor_costo_global, mejor_plan_global) # Imprimir resultados al alcanzar límite de tiempo
            return
        
        plan_dias_actual = {d: [] for d in range(dias_a_probar_fuerza_bruta)}
        conteo_tomas_por_dia = [0] * dias_a_probar_fuerza_bruta
        es_asignacion_valida = True
        for indice_toma, dia_asignado in enumerate(asignacion_actual_tupla):
            plan_dias_actual[dia_asignado].append(indice_toma)
            conteo_tomas_por_dia[dia_asignado] += 1
            if conteo_tomas_por_dia[dia_asignado] > maximo_tomas_por_dia:
                es_asignacion_valida = False
                break
        if not es_asignacion_valida:
            continue
        costo_actual = calcular_costo_planificacion(plan_dias_actual, matriz_actores)
        if costo_actual < mejor_costo_global:
            mejor_costo_global = costo_actual
            mejor_plan_global = {dia: lista_tomas[:] for dia, lista_tomas in plan_dias_actual.items()}

    print("\n<---***--- Todas las combinaciones posibles han sido exploradas. ---***--->")
    imprimir_resultados_finales(mejor_costo_global, mejor_plan_global) 

    establecer_bandera_finalizado()
    guardar_checkpoint(mejor_costo_global, mejor_plan_global, contador_iteraciones_ciclo, iteracion_inicio_global)

print("<---***--- Incia algoritmo de fuerza bruta ---***--->")
ejecutar_fuerza_bruta(tiempo_maximo_ejecucion_por_ciclo)

<---***--- Incia algoritmo de fuerza bruta ---***--->

Iniciando Fuerza Bruta para 10 días con 10 tomas.
Mejor costo inicial: inf (si se cargó un checkpoint)
Tiempo máximo por ciclo de ejecución: 3600 segundos
Avanzando el iterador hasta la iteración 0...

--- Límite de tiempo (3600s) alcanzado en este ciclo. Guardando checkpoint. ---

<---***--- Checkpoint guardado en la iteración 96105410. Mejor costo: 14 ---***--->

--- Resultados finales ---
Mejor costo encontrado: 14
Planificación final:
  Día 0: Tomas [0, 1, 2, 4, 6, 7] (Cantidad de Tomas: 6)
  Día 1: Tomas [3, 5, 8, 9] (Cantidad de Tomas: 4)
  Día 2: Tomas [] (Cantidad de Tomas: 0)
  Día 3: Tomas [] (Cantidad de Tomas: 0)
  Día 4: Tomas [] (Cantidad de Tomas: 0)
  Día 5: Tomas [] (Cantidad de Tomas: 0)
  Día 6: Tomas [] (Cantidad de Tomas: 0)
  Día 7: Tomas [] (Cantidad de Tomas: 0)
  Día 8: Tomas [] (Cantidad de Tomas: 0)
  Día 9: Tomas [] (Cantidad de Tomas: 0)


# Algoritmo "Fuerza Bruta" CON PuLP (Segun sugerencia del profesor)

In [21]:
cantidad_tomas_PULP = matriz_actores_original.shape[0]
cantidad_actores_PULP = matriz_actores_original.shape[1]
maximo_tomas_por_dia_PULP = 6

maximo_dias_a_considerar_PULP = int((cantidad_tomas_PULP / maximo_tomas_por_dia_PULP) * 2) 

print("--- Configuración del Problema ---")
print(f"Número de Tomas: {cantidad_tomas_PULP}")
print(f"Número de Actores: {cantidad_actores_PULP}")
print(f"Máximo de Tomas por Día: {maximo_tomas_por_dia_PULP}")
print(f"Máximo de Días a Considerar: {maximo_dias_a_considerar_PULP}\n")

problema_optimizacion = LpProblem("Programacion_Tomas_Pelicula", LpMinimize)
indices_tomas = list(range(cantidad_tomas_PULP))
indices_dias = list(range(maximo_dias_a_considerar_PULP))
indices_actores = list(range(cantidad_actores_PULP))
variable_toma_en_dia = LpVariable.dicts("Toma_en_Dia", (indices_tomas, indices_dias), 0, 1, LpBinary)
variable_actor_necesario_dia = LpVariable.dicts("Actor_Necesario_Dia", (indices_actores, indices_dias), 0, 1, LpBinary)
problema_optimizacion += lpSum(variable_actor_necesario_dia[a][d] for a in indices_actores for d in indices_dias), "Costo_Total_Actores"

for t in indices_tomas:
    problema_optimizacion += lpSum(variable_toma_en_dia[t][d] for d in indices_dias) == 1, f"R1_Toma_{t}_Asignada_Una_Vez"

for d in indices_dias:
    problema_optimizacion += lpSum(variable_toma_en_dia[t][d] for t in indices_tomas) <= maximo_tomas_por_dia_PULP, f"R2_Max_Tomas_Dia_{d}"

valor_grande = cantidad_tomas_PULP
for a in indices_actores:
    for d in indices_dias:
        problema_optimizacion += lpSum(matriz_actores_original[t][a] * variable_toma_en_dia[t][d] for t in indices_tomas) <= valor_grande * variable_actor_necesario_dia[a][d], f"R3_Actor_{a}_Necesario_Dia_{d}"

print("Resolviendo el problema con PuLP...")
tiempo_inicio_PULP = time.time()

problema_optimizacion.solve(PULP_CBC_CMD(timeLimit=600))

tiempo_fin_PULP = time.time()
duracion_ejecucion_PULP = tiempo_fin_PULP - tiempo_inicio_PULP

print("\n--- Resultados de la Optimización ---")
print(f"Estado de la solución: {LpStatus[problema_optimizacion.status]}")
print(f"Costo óptimo de actores: {value(problema_optimizacion.objective)}")
print(f"Tiempo de ejecución del solver: {duracion_ejecucion_PULP:.2f} segundos")

if problema_optimizacion.status == 1 or problema_optimizacion.status == 2:
    print("\n--- Distribución de Tomas por Día ---")

    solucion_obtenida = {}
    total_dias_utilizados_PULP = 0

    for d in indices_dias:
        tomas_en_dia_actual = []
        for t in indices_tomas:
            if variable_toma_en_dia[t][d].varValue > 0.5:
                tomas_en_dia_actual.append(t)
        if tomas_en_dia_actual:
            total_dias_utilizados_PULP += 1
            solucion_obtenida[d] = sorted(tomas_en_dia_actual)
            actores_presentes_este_dia = np.zeros(cantidad_actores_PULP, dtype=int)
            for indice_toma_actual in tomas_en_dia_actual:
                actores_presentes_este_dia = np.logical_or(actores_presentes_este_dia, matriz_actores_original[indice_toma_actual, :])
            conteo_actores_dia = int(np.sum(actores_presentes_este_dia))
            print(f"\n Día {d} (Tomas: {len(tomas_en_dia_actual)} / Actores únicos: {conteo_actores_dia}):")
            print(f"    Tomas asignadas: {tomas_en_dia_actual}")
    print(f"\nTotal de días utilizados en la solución: {total_dias_utilizados_PULP}")
else:
    print("\n No se encontro solucion optima")

--- Configuración del Problema ---
Número de Tomas: 30
Número de Actores: 10
Máximo de Tomas por Día: 6
Máximo de Días a Considerar: 10

Resolviendo el problema con PuLP...

--- Resultados de la Optimización ---
Estado de la solución: Optimal
Costo óptimo de actores: 27.0
Tiempo de ejecución del solver: 600.50 segundos

--- Distribución de Tomas por Día ---

 Día 2 (Tomas: 6 / Actores únicos: 4):
    Tomas asignadas: [1, 12, 19, 26, 27, 29]

 Día 5 (Tomas: 6 / Actores únicos: 6):
    Tomas asignadas: [4, 7, 8, 15, 20, 24]

 Día 6 (Tomas: 6 / Actores únicos: 3):
    Tomas asignadas: [13, 16, 17, 18, 22, 23]

 Día 8 (Tomas: 6 / Actores únicos: 7):
    Tomas asignadas: [0, 2, 3, 5, 10, 14]

 Día 9 (Tomas: 6 / Actores únicos: 7):
    Tomas asignadas: [6, 9, 11, 21, 25, 28]

Total de días utilizados en la solución: 5


# Algoritmo genetico (Heuristico)

In [48]:
cantidad_actores_AG = matriz_actores_original.shape[1]
cantidad_tomas_AG = matriz_actores_original.shape[0]
maximo_tomas_por_dia_AG = 6
ruta_checkpoint_AG = 'data/algoritmo_genetico_checkpoint.json'
ruta_bandera_terminado_AG = 'data/algoritmo_genetico_terminado.flag'

def calcular_costo_dia(tomas_del_dia_actual):
    if not tomas_del_dia_actual:
        return 0
    actores_requeridos_por_toma_ag = matriz_actores_original[list(tomas_del_dia_actual), :]
    actores_presentes_ag = np.sum(actores_requeridos_por_toma_ag, axis=0) > 0
    return int(np.sum(actores_presentes_ag))

def evaluar_individuo(individuo_actual):
    dias_organizados = {}
    for indice_toma_ag, indice_dia_ag in enumerate(individuo_actual):
        if indice_dia_ag not in dias_organizados:
            dias_organizados[indice_dia_ag] = []
        dias_organizados[indice_dia_ag].append(indice_toma_ag)
    costo_total_individual = 0
    penalizacion = 0
    for indice_dia_ag_ordenado in sorted(dias_organizados.keys()):
        tomas_en_el_dia_actual = dias_organizados[indice_dia_ag_ordenado]
        if len(tomas_en_el_dia_actual) > maximo_tomas_por_dia_AG:
            penalizacion += (len(tomas_en_el_dia_actual) - maximo_tomas_por_dia_AG) * 10
        costo_total_individual += calcular_costo_dia(tomas_en_el_dia_actual)
    return 1 / (costo_total_individual + penalizacion + 1)

def crear_poblacion_inicial(tamano_poblacion):
    poblacion = []
    for _ in range(tamano_poblacion):
        individuo = [random.randint(0, cantidad_tomas_AG // maximo_tomas_por_dia_AG + 2) for _ in range(cantidad_tomas_AG)]
        poblacion.append(individuo)
    return poblacion

def mutar(individuo_a_mutar, tasa_mutacion):
    for i in range(len(individuo_a_mutar)):
        if random.random() < tasa_mutacion:
            dias_existentes = set(individuo_a_mutar)
            if dias_existentes:
                max_indice_dia = max(dias_existentes)
            else:
                max_indice_dia = 0
            individuo_a_mutar[i] = random.randint(0, max_indice_dia + 2)
    return individuo_a_mutar

def cruzar(padre1, padre2):
    hijo1 = []
    hijo2 = []
    for i in range(len(padre1)):
        if random.random() < 0.5:
            hijo1.append(padre1[i])
            hijo2.append(padre2[i])
        else:
            hijo1.append(padre2[i])
            hijo2.append(padre1[i])
    return hijo1, hijo2

def obtener_costo_desde_aptitud(aptitud):
    if aptitud == 0:
        return float('inf')
    return (1 / aptitud) - 1

def particionar_a_dias_reales(individuo_a_particionar):
    diccionario_dias = {}
    for indice_toma_ag, asignacion_dia_ag in enumerate(individuo_a_particionar):
        if asignacion_dia_ag not in diccionario_dias:
            diccionario_dias[asignacion_dia_ag] = []
        diccionario_dias[asignacion_dia_ag].append(indice_toma_ag)
    particion_real = [sorted(diccionario_dias[indice_dia_ag]) for indice_dia_ag in sorted(diccionario_dias.keys())]
    return particion_real

def guardar_checkpoint_ag(generacion_actual, mejor_costo_actual, mejor_particion_actual, tiempo_total_ag_acumulado, poblacion_actual):
    costo_a_guardar = mejor_costo_actual
    if mejor_costo_actual == float('inf'):
        costo_a_guardar = "Infinity"
    else:
        costo_a_guardar = int(mejor_costo_actual)
    datos_ag = {
        'generacion': generacion_actual,
        'mejor_costo': costo_a_guardar,
        'mejor_particion': mejor_particion_actual,
        'tiempo_total': tiempo_total_ag_acumulado,
        'poblacion': poblacion_actual,
        'terminado': False
    }
    with open(ruta_checkpoint_AG, 'w') as f:
        json.dump(datos_ag, f)
    print(f"\n--- Checkpoint AG guardado en {ruta_checkpoint_AG} ---")

def cargar_checkpoint_ag():
    if os.path.exists(ruta_checkpoint_AG):
        try:
            with open(ruta_checkpoint_AG, 'r') as f:
                datos_ag = json.load(f)
            if datos_ag.get('mejor_costo') == "Infinity":
                datos_ag['mejor_costo'] = float('inf')
            print(f"\n--- Checkpoint AG cargado desde {ruta_checkpoint_AG} ---")
            return datos_ag
        except json.JSONDecodeError:
            print(f"Error al leer el archivo de checkpoint AG '{ruta_checkpoint_AG}'. Puede estar corrupto.")
            os.remove(ruta_checkpoint_AG)
            print(f"Archivo de checkpoint AG '{ruta_checkpoint_AG}' corrupto eliminado. Iniciando desde cero.")
            return None
    return None

def establecer_bandera_terminado_ag():
    with open(ruta_bandera_terminado_AG, 'w') as f:
        f.write("TERMINATED")
    print(f"\n--- Bandera de terminación AG creada: {ruta_bandera_terminado_AG} ---")

def esta_terminado_ag():
    return os.path.exists(ruta_bandera_terminado_AG)

def ejecutar_algoritmo_genetico(tamano_poblacion_ag, tasa_mutacion_ag, generaciones_ag, tiempo_maximo_ejecucion_por_ciclo_ag):
    mejor_costo_global_ag = float('inf')
    mejor_particion_global_ag = []
    tiempo_total_acumulado_ag = 0.0
    generacion_actual_ag = 0
    poblacion_ag = []
    print("\nIniciando Algoritmo Genético...")
    if esta_terminado_ag():
        print("\nEl algoritmo genético ya fue ejecutado y finalizó por completo en una sesión anterior.")
        datos_checkpoint_ag = cargar_checkpoint_ag()
        if datos_checkpoint_ag and datos_checkpoint_ag.get('terminado', False):
            print(f"Mejor costo final: {datos_checkpoint_ag['mejor_costo']}")
            print(f"Partición óptima final: {datos_checkpoint_ag['mejor_particion']}")
            print(f"Tiempo total final: {datos_checkpoint_ag['tiempo_total']:.2f} s")
            print(f"Generaciones totales ejecutadas: {datos_checkpoint_ag['generacion']}")
        return
    datos_checkpoint_ag = cargar_checkpoint_ag()
    if datos_checkpoint_ag:
        generacion_actual_ag = datos_checkpoint_ag['generacion']
        mejor_costo_global_ag = datos_checkpoint_ag['mejor_costo']
        mejor_particion_global_ag = datos_checkpoint_ag['mejor_particion']
        tiempo_total_acumulado_ag = datos_checkpoint_ag['tiempo_total']
        poblacion_ag = datos_checkpoint_ag['poblacion']
        print(f"Reanudando desde checkpoint: Generación {generacion_actual_ag}, Mejor costo = {mejor_costo_global_ag}, Tiempo total acumulado = {tiempo_total_acumulado_ag:.2f} s")
    else:
        poblacion_ag = crear_poblacion_inicial(tamano_poblacion_ag)
        print(f"Población inicial creada con {len(poblacion_ag)} individuos.")
    tiempo_inicio_bloque_actual_ag = time.time()
    try:
        while generacion_actual_ag < generaciones_ag and (time.time() - tiempo_inicio_bloque_actual_ag < tiempo_maximo_ejecucion_por_ciclo_ag):
            poblacion_evaluada = []
            for ind_eval in poblacion_ag:
                aptitud = evaluar_individuo(ind_eval)
                poblacion_evaluada.append((aptitud, ind_eval))
            poblacion_evaluada.sort(key=lambda x: x[0], reverse=True)
            mejor_aptitud_actual = poblacion_evaluada[0][0]
            mejor_individuo_actual = poblacion_evaluada[0][1]
            costo_actual_obtenido = obtener_costo_desde_aptitud(mejor_aptitud_actual)
            if costo_actual_obtenido < mejor_costo_global_ag:
                mejor_costo_global_ag = costo_actual_obtenido
                mejor_particion_global_ag = particionar_a_dias_reales(mejor_individuo_actual)
            cantidad_elite = int(tamano_poblacion_ag * 0.1)
            nueva_poblacion_ag = [ind_elite for apt, ind_elite in poblacion_evaluada[:cantidad_elite]]
            while len(nueva_poblacion_ag) < tamano_poblacion_ag:
                candidatos1 = random.sample(poblacion_evaluada, min(5, len(poblacion_evaluada)))
                padre1_apt, padre1 = max(candidatos1, key=lambda x: x[0])
                candidatos2 = random.sample(poblacion_evaluada, min(5, len(poblacion_evaluada)))
                padre2_apt, padre2 = max(candidatos2, key=lambda x: x[0])
                hijo1, hijo2 = cruzar(padre1, padre2)
                hijo1 = mutar(hijo1, tasa_mutacion_ag)
                hijo2 = mutar(hijo2, tasa_mutacion_ag)
                nueva_poblacion_ag.append(hijo1)
                if len(nueva_poblacion_ag) < tamano_poblacion_ag:
                    nueva_poblacion_ag.append(hijo2)
            poblacion_ag = nueva_poblacion_ag
            generacion_actual_ag += 1
        tiempo_transcurrido_bloque_actual_ag = time.time() - tiempo_inicio_bloque_actual_ag
        tiempo_total_acumulado_ag += tiempo_transcurrido_bloque_actual_ag
        if generacion_actual_ag >= generaciones_ag or time.time() - tiempo_inicio_bloque_actual_ag >= tiempo_maximo_ejecucion_por_ciclo_ag:
            print("\nAlgoritmo Genético finalizado o tiempo de ejecución agotado para este bloque.")
            guardar_checkpoint_ag(generacion_actual_ag, mejor_costo_global_ag, mejor_particion_global_ag,
                                  tiempo_total_acumulado_ag, poblacion_ag)
            if generacion_actual_ag >= generaciones_ag:
                establecer_bandera_terminado_ag()
                print(f"\n--- Algoritmo Genético completado después de {generacion_actual_ag} generaciones ---")
                final_datos_checkpoint_ag = {
                    'generacion': generacion_actual_ag,
                    'mejor_costo': int(mejor_costo_global_ag) if mejor_costo_global_ag != float('inf') else "Infinity",
                    'mejor_particion': mejor_particion_global_ag,
                    'tiempo_total': tiempo_total_acumulado_ag,
                    'poblacion': poblacion_ag,
                    'terminado': True
                }
                with open(ruta_checkpoint_AG, 'w') as f:
                    json.dump(final_datos_checkpoint_ag, f)
                print(f"Resultados finales guardados en {ruta_checkpoint_AG}")

        print("\n--- Resumen de la Ejecución Actual del Algoritmo Genético ---")
        print(f"Generación actual: {generacion_actual_ag}")
        print(f"Mejor costo encontrado hasta ahora: {mejor_costo_global_ag}")
        print(f"Partición óptima hasta ahora: {mejor_particion_global_ag}")
        print(f"Tiempo en este flujo: {tiempo_transcurrido_bloque_actual_ag:.2f} s")
        print(f"Tiempo total acumulado: {tiempo_total_acumulado_ag:.2f} s")

    except Exception as e:
        print(f"Se produjo un error durante la ejecución del Algoritmo Genético: {e}")
        tiempo_transcurrido_bloque_actual_ag = time.time() - tiempo_inicio_bloque_actual_ag
        tiempo_total_acumulado_ag += tiempo_transcurrido_bloque_actual_ag
        guardar_checkpoint_ag(generacion_actual_ag, mejor_costo_global_ag, mejor_particion_global_ag,
                              tiempo_total_acumulado_ag, poblacion_ag)
        print(f"\n--- Checkpoint AG guardado debido a un error en {ruta_checkpoint_AG} ---")

        print("\n--- Resumen de la Ejecución (debido a error en AG) ---")
        print(f"Generación actual: {generacion_actual_ag}")
        print(f"Mejor costo encontrado hasta ahora: {mejor_costo_global_ag}")
        print(f"Partición óptima hasta ahora: {mejor_particion_global_ag}")
        print(f"Tiempo en este flujo: {tiempo_transcurrido_bloque_actual_ag:.2f} s")
        print(f"Tiempo total acumulado: {tiempo_total_acumulado_ag:.2f} s")

In [49]:
#Configuracion de algoritmo genetico
tamano_poblacion_ag = 2000  # Número de individuos en cada generación
tasa_mutacion_ag = 0.08 
generaciones_ag = 100000 # Número máximo de generaciones (si no se alcanza el tiempo límite)
tiempo_maximo_ejecucion_por_ciclo_ag = 3600 # Tiempo máximo en segundos por bloque de ejecución 3600 =  1 hora

ejecutar_algoritmo_genetico(
    tamano_poblacion_ag=tamano_poblacion_ag,
    tasa_mutacion_ag=tasa_mutacion_ag,
    generaciones_ag=generaciones_ag,
    tiempo_maximo_ejecucion_por_ciclo_ag=tiempo_maximo_ejecucion_por_ciclo_ag
)


Iniciando Algoritmo Genético...
Población inicial creada con 2000 individuos.

Algoritmo Genético finalizado o tiempo de ejecución agotado para este bloque.

--- Checkpoint AG guardado en data/algoritmo_genetico_checkpoint.json ---

--- Resumen de la Ejecución Actual del Algoritmo Genético ---
Generación actual: 13232
Mejor costo encontrado hasta ahora: 27.0
Partición óptima hasta ahora: [[0, 1, 11, 12, 19, 21], [3, 4, 14, 15, 24, 29], [13, 16, 17, 18, 22, 23], [7, 9, 10, 20, 25, 28], [2, 5, 6, 8, 26, 27]]
Tiempo en este flujo: 3600.39 s
Tiempo total acumulado: 3600.39 s
